In [241]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import ndcg_score

In [242]:
df = pd.read_csv("train_df.csv").drop_duplicates()
df_test = pd.read_csv("test_df.csv").drop_duplicates()
df

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
0,758,9,0,0,1,20,3,40,0,3,...,0.204682,0.271755,0.055623,0,0,0,0.38648,0.0,0.0,0
1,758,9,0,0,1,20,3,40,0,3,...,0.195531,0.188787,0.036914,0,0,0,0.10982,0.0,0.0,0
2,758,9,0,0,1,20,3,40,0,3,...,0.148609,0.186517,0.027718,0,0,0,0.03674,0.0,0.0,0
3,758,9,0,0,1,20,3,40,0,3,...,0.223748,0.229039,0.051247,0,0,0,0.00000,0.0,0.0,0
4,758,9,0,0,1,20,3,40,0,3,...,0.170935,0.249031,0.042568,0,0,0,0.00000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,494693,9,0,0,0,9,4,38,6,6,...,0.309672,0.921060,0.285226,0,0,0,0.98807,0.0,0.0,0
15077,494693,9,0,0,0,9,4,38,6,6,...,0.303805,0.995086,0.302312,0,0,0,0.87146,0.0,0.0,0
15078,494693,9,0,0,0,9,4,38,6,6,...,0.346538,0.993070,0.344137,0,0,0,0.49999,0.0,0.0,0
15079,494693,9,0,0,0,9,4,38,6,6,...,0.243154,0.994833,0.241898,0,0,0,0.67614,0.0,0.0,0


In [243]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)
df = df.sort_values(by='search_id')

In [244]:
def make_dataset(qids):
    Data = df.loc[np.isin(df.search_id, qids)]
    return Data[df.columns[1:-1]], Data.target, Data.search_id

In [245]:
qid = df.search_id
X = df[df.columns[1:-1]]
y = df.target
qid_train, qid_val = train_test_split(qid.unique(), test_size=0.20)

In [246]:
X_train, y_train, Qid_train = make_dataset(qid_train)
X_val, y_val, Qid_val = make_dataset(qid_val)

In [247]:
ranker = xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:ndcg", lambdarank_pair_method="topk")
ranker.fit(X_train, y_train, qid=Qid_train, 
           eval_set=[(X_val, y_val)],
           eval_qid=[Qid_val],
           eval_metric=["ndcg"],
           verbose=True,
)

[0]	validation_0-ndcg:0.88515
[1]	validation_0-ndcg:0.89283
[2]	validation_0-ndcg:0.90028
[3]	validation_0-ndcg:0.90040
[4]	validation_0-ndcg:0.90285
[5]	validation_0-ndcg:0.89833
[6]	validation_0-ndcg:0.89722
[7]	validation_0-ndcg:0.89806
[8]	validation_0-ndcg:0.89771
[9]	validation_0-ndcg:0.90039
[10]	validation_0-ndcg:0.89703
[11]	validation_0-ndcg:0.89721
[12]	validation_0-ndcg:0.89889
[13]	validation_0-ndcg:0.89792
[14]	validation_0-ndcg:0.89758
[15]	validation_0-ndcg:0.89934
[16]	validation_0-ndcg:0.89702
[17]	validation_0-ndcg:0.89875
[18]	validation_0-ndcg:0.89763
[19]	validation_0-ndcg:0.89801
[20]	validation_0-ndcg:0.89798
[21]	validation_0-ndcg:0.90171
[22]	validation_0-ndcg:0.90086
[23]	validation_0-ndcg:0.89871
[24]	validation_0-ndcg:0.90195
[25]	validation_0-ndcg:0.89724
[26]	validation_0-ndcg:0.89626
[27]	validation_0-ndcg:0.89486
[28]	validation_0-ndcg:0.89493
[29]	validation_0-ndcg:0.89273
[30]	validation_0-ndcg:0.89385
[31]	validation_0-ndcg:0.89364
[32]	validation_0-

/Users/daniel/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[37]	validation_0-ndcg:0.89390
[38]	validation_0-ndcg:0.89327
[39]	validation_0-ndcg:0.89337
[40]	validation_0-ndcg:0.89543
[41]	validation_0-ndcg:0.89382
[42]	validation_0-ndcg:0.89152
[43]	validation_0-ndcg:0.89186
[44]	validation_0-ndcg:0.89224
[45]	validation_0-ndcg:0.89099
[46]	validation_0-ndcg:0.88926
[47]	validation_0-ndcg:0.88678
[48]	validation_0-ndcg:0.89018
[49]	validation_0-ndcg:0.88987
[50]	validation_0-ndcg:0.89106
[51]	validation_0-ndcg:0.89155
[52]	validation_0-ndcg:0.89070
[53]	validation_0-ndcg:0.89070
[54]	validation_0-ndcg:0.88986
[55]	validation_0-ndcg:0.88971
[56]	validation_0-ndcg:0.88979
[57]	validation_0-ndcg:0.89051
[58]	validation_0-ndcg:0.89030
[59]	validation_0-ndcg:0.89078
[60]	validation_0-ndcg:0.89078
[61]	validation_0-ndcg:0.88980
[62]	validation_0-ndcg:0.89168
[63]	validation_0-ndcg:0.89202
[64]	validation_0-ndcg:0.89030
[65]	validation_0-ndcg:0.88811
[66]	validation_0-ndcg:0.89019
[67]	validation_0-ndcg:0.88978
[68]	validation_0-ndcg:0.88978
[69]	val

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='topk',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

# Считаем метрику на тесте

In [248]:
df_test = df_test.sort_values(by='search_id')

In [249]:
X_test = df_test[df_test.columns[1:-1]]
X_test["qid"] = df_test.search_id
y_test = df_test.target
print(ranker.score(X_test, y_test))

0.9204290702469636


/var/folders/cz/1v3n7t6j6zs9js7m7v_t9ttc0000gn/T/ipykernel_6512/2730232665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["qid"] = df_test.search_id
